# Conversion of forcing created for MESH to HYPE nc or txt format for model simulation.
## nc creation is based on this page:
http://www.smhi.net/hype/wiki/doku.php?id=start:hype_file_reference:nnobs.nc

In [1]:
# packages are loaded
import xarray as xr
import glob
import netCDF4 as nc4
import os

In [2]:
# Set the folder path where the remapped .nc file is located for MESH (it can be any remapped nc file)
nc_file_source = '/home/shg096/scratch/West/mesh_model/MESH_forcing.nc'
path_to_save = '/home/shg096/scratch/West/HYPE/'

In [3]:
# reading the nc file
ds = xr.open_dataset(nc_file_source)

# rename the subbasin or other dimension name to id based on HYPE instruction
ds = ds.rename({'subbasin': 'id'})
ds = ds.set_coords(['lat', 'lon'])

# drop some of the variables that are not used
ds = ds.drop_vars(['RDRS_v2.1_P_UVC_10m',\
                   'RDRS_v2.1_P_FI_SFC',\
                   'RDRS_v2.1_P_FB_SFC',\
                   'RDRS_v2.1_P_HU_1.5m',\
                   'RDRS_v2.1_P_P0_SFC',\
                   'crs']) # ,'lat','lon'

# change time from UTC to approximate local time
time_diff = -7 # Alberta, BC

# roll the time based on hour of difference to have more accurate 
ds['time'] = ds['time'].roll(time=time_diff)

# Remove the first or last roll_steps time steps
if time_diff < 0:
    ds = ds.isel(time=slice( None, time_diff))
elif time_diff > 0:
    ds = ds.isel(time=slice( time_diff, None))

# sample to daily
ds_daily = ds.resample(time='D').mean() # to create the xarray dataframe with daily time
ds_daily['TMAXobs'] = ds['RDRS_v2.1_P_TT_1.5m'].resample(time='D').max() - 273.15 # kelvin to C
ds_daily['TMINobs'] = ds['RDRS_v2.1_P_TT_1.5m'].resample(time='D').min() - 273.15 # kelvin to C
ds_daily['Tobs'] = ds['RDRS_v2.1_P_TT_1.5m'].resample(time='D').mean() - 273.15 # kelvin to C
ds_daily['Pobs'] = ds['RDRS_v2.1_P_PR0_SFC'].resample(time='D').sum() * 3600 # mm/s to mm/day 

ds_daily = ds_daily.drop_vars(['RDRS_v2.1_P_PR0_SFC',\
                               'RDRS_v2.1_P_TT_1.5m'])

## save as nc files

In [4]:
# create the path to save if not exists
if not os.path.isdir(path_to_save):
    os.makedirs(path_to_save)

# Tmax
os.remove(path_to_save+'TMAXobs.nc')
TMAXobs = ds_daily.copy()
TMAXobs = TMAXobs.drop_vars(['TMINobs','Tobs','Pobs'])
TMAXobs.to_netcdf(path_to_save+'TMAXobs.nc', encoding = {'TMAXobs':{'_FillValue': -9999}})
TMAXobs.to_netcdf(path_to_save+'TMAXobs.nc')

# Tmin
os.remove(path_to_save+'TMINobs.nc')
TMINobs = ds_daily.copy()
TMINobs = TMINobs.drop_vars(['TMAXobs','Tobs','Pobs'])
TMINobs.to_netcdf(path_to_save+'TMINobs.nc', encoding = {'TMINobs':{'_FillValue': -9999}})
TMINobs.to_netcdf(path_to_save+'TMINobs.nc')

# T
os.remove(path_to_save+'Tobs.nc')
Tobs = ds_daily.copy()
Tobs = Tobs.drop_vars(['TMAXobs','TMINobs','Pobs'])
Tobs.to_netcdf(path_to_save+'Tobs.nc', encoding = {'Tobs':{'_FillValue': -9999}})
Tobs.to_netcdf(path_to_save+'Tobs.nc')

# P
os.remove(path_to_save+'Pobs.nc')
Pobs = ds_daily.copy()
Pobs = Pobs.drop_vars(['TMAXobs','TMINobs','Tobs'])
Pobs.to_netcdf(path_to_save+'Pobs.nc', encoding = {'Pobs':{'_FillValue': -9999}})
Pobs.to_netcdf(path_to_save+'Pobs.nc')

## save as txt file for input if needed

In [5]:
ds_daily = ds_daily.drop_vars(['lat','lon'])

ds_daily_df_Tmax = ds_daily['TMAXobs'].to_dataframe()
ds_daily_df_Tmax = ds_daily_df_Tmax.unstack()
ds_daily_df_Tmax = ds_daily_df_Tmax.T
ds_daily_df_Tmax = ds_daily_df_Tmax.droplevel(level=0, axis=0)
ds_daily_df_Tmax.columns.name = None
ds_daily_df_Tmax.index.name = 'Date'
ds_daily_df_Tmax
ds_daily_df_Tmax.to_csv(path_to_save+'TMAXobs.txt',\
                        sep='\t', na_rep='', index_label='time', float_format='%.1f')

ds_daily_df_Tmin = ds_daily['TMINobs'].to_dataframe()
ds_daily_df_Tmin = ds_daily_df_Tmin.unstack()
ds_daily_df_Tmin = ds_daily_df_Tmin.T
ds_daily_df_Tmin = ds_daily_df_Tmin.droplevel(level=0, axis=0)
ds_daily_df_Tmin.columns.name = None
ds_daily_df_Tmin.index.name = 'Date'
ds_daily_df_Tmin
ds_daily_df_Tmin.to_csv(path_to_save+'TMINobs.txt',\
                        sep='\t', na_rep='', index_label='time', float_format='%.1f')

ds_daily_df_Tmean = ds_daily['Tobs'].to_dataframe()
ds_daily_df_Tmean = ds_daily_df_Tmean.unstack()
ds_daily_df_Tmean = ds_daily_df_Tmean.T
ds_daily_df_Tmean = ds_daily_df_Tmean.droplevel(level=0, axis=0)
ds_daily_df_Tmean.columns.name = None
ds_daily_df_Tmean.index.name = 'Date'
ds_daily_df_Tmean
ds_daily_df_Tmean.to_csv(path_to_save+'Tobs.txt',\
                         sep='\t', na_rep='', index_label='time', float_format='%.1f')

ds_daily_df_Pobs = ds_daily['Pobs'].to_dataframe()
ds_daily_df_Pobs = ds_daily_df_Pobs.unstack()
ds_daily_df_Pobs = ds_daily_df_Pobs.T
ds_daily_df_Pobs = ds_daily_df_Pobs.droplevel(level=0, axis=0)
ds_daily_df_Pobs.columns.name = None
ds_daily_df_Pobs.index.name = 'Date'
ds_daily_df_Pobs
ds_daily_df_Pobs.to_csv(path_to_save+'Pobs.txt',\
                         sep='\t', na_rep='', index_label='time', float_format='%.1f')


In [6]:
# EXTRA code for generating a few days of input for testing a set up
# 
# List of column names
# columns = list (landcover_type_prepared['subid'])

# # Start date and end date
# start_date = datetime.date(2023, 10, 1)
# end_date = start_date + datetime.timedelta(days=9)  # 10 days

# # List of dates for 10 days
# dates = [start_date + datetime.timedelta(days=i) for i in range(10)]

# # Create an empty DataFrame
# df = pd.DataFrame(columns=columns, index=dates)

# df = df.round(3)

# df.index.name = 'Date'

# ##
# for col in columns:
#     df[col] = np.random.uniform(0, 1, size=len(dates))
# df.to_csv(path_to_save+'Pobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.2f')

# for i, col in enumerate(columns):
#     # Generate values from -5 to +5 for each column
#     df[col] = [(i % 11) - 5 for i in range(len(dates))]
# df.to_csv(path_to_save+'Tobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.1f')

# for col in columns:
#     df[col] = df[col]+1
# df.to_csv(path_to_save+'TMAXobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.1f')

# for col in columns:
#     df[col] = df[col]-2
# df.to_csv(path_to_save+'TMINobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.1f')

In [7]:
import xarray as xr
ds = xr.open_dataset(path_to_save+'TMAXobs.nc')
ds
import netCDF4 as nc4

ncid = nc4.Dataset(path_to_save+'TMAXobs.nc')
ncid['TMAXobs'][:]

masked_array(
  data=[[-19.76161954,  -7.8089399 ,  -9.97479155, ..., -19.90054982,
         -18.02817686, -23.21726809],
        [-17.35760057,  -7.66104934,  -9.96431556, ..., -19.38495723,
         -17.74385849, -22.94001826],
        [-15.84581261,  -8.02279789, -10.15236716, ..., -19.6420587 ,
         -18.01059101, -23.25255864],
        ...,
        [-13.43259172,  -5.85975205,  -1.03139569, ..., -19.20718218,
         -21.62915074, -22.99387594],
        [ -9.24669858,   0.96465074,  -1.44407905, ..., -21.4311276 ,
         -20.44699877, -23.53324109],
        [-11.76461008,  -0.43071804,  -2.08934352, ..., -16.72650111,
         -17.60499673, -21.32597125]],
  mask=False,
  fill_value=1e+20)

In [11]:

import pandas as pd
from   easymore import Easymore
esmr = Easymore()

ds = esmr.dataframe_to_netcdf_xr(ds_daily_df_Tmax,
                                 data_frame_DateTime_column = 'Date',
                                 variable_name = 'TMAXobs',
                                 variable_dim_name = 'id',
                                 unit_of_variable = 'C',
                                 variable_long_name = 'maximume daily temprature',
                                 Fill_value = -9999.0,
                                 nc_file_name = path_to_save+'TMAXobs.nc')

ds = esmr.dataframe_to_netcdf_xr(ds_daily_df_Tmin,
                                 data_frame_DateTime_column = 'Date',
                                 variable_name = 'TMINobs',
                                 variable_dim_name = 'id',
                                 unit_of_variable = 'C',
                                 variable_long_name = 'minimume daily temprature',
                                 Fill_value = -9999.0,
                                 nc_file_name = path_to_save+'TMINobs.nc')

ds = esmr.dataframe_to_netcdf_xr(ds_daily_df_Tmean,
                                 data_frame_DateTime_column = 'Date',
                                 variable_name = 'Tobs',
                                 variable_dim_name = 'id',
                                 unit_of_variable = 'C',
                                 variable_long_name = 'mean daily temprature',
                                 Fill_value = -9999.0,
                                 nc_file_name = path_to_save+'Tobs.nc')

ds = esmr.dataframe_to_netcdf_xr(ds_daily_df_Pobs,
                                 data_frame_DateTime_column = 'Date',
                                 variable_name = 'Pobs',
                                 variable_dim_name = 'id',
                                 unit_of_variable = 'mm',
                                 variable_long_name = 'Precipitation',
                                 Fill_value = -9999.0,
                                 nc_file_name = path_to_save+'Pobs.nc')


EASYMORE version 2.0.0 is initiated.
EASYMORE detects that the input datafarame is pandas dataframe
EASYMORE detects that index is pandas datatime
EASYMORE saved the nc file here:  /home/shg096/scratch/West/HYPE/TMAXobs.nc
EASYMORE detects that the input datafarame is pandas dataframe
EASYMORE detects that index is pandas datatime
EASYMORE saved the nc file here:  /home/shg096/scratch/West/HYPE/TMINobs.nc
EASYMORE detects that the input datafarame is pandas dataframe
EASYMORE detects that index is pandas datatime
EASYMORE saved the nc file here:  /home/shg096/scratch/West/HYPE/Tobs.nc
EASYMORE detects that the input datafarame is pandas dataframe
EASYMORE detects that index is pandas datatime
EASYMORE saved the nc file here:  /home/shg096/scratch/West/HYPE/Pobs.nc


In [9]:
import netCDF4 as nc4



masked_array(
  data=[[1.25906552e+00, 1.23255970e+01, 4.52812427e+01, ...,
         6.30789444e+00, 1.03934755e+01, 3.76399188e+00],
        [1.67402785e+00, 1.38089692e+01, 2.24481744e+01, ...,
         7.58311958e+00, 1.17308854e+01, 4.32474035e+00],
        [1.04241813e+00, 1.09317900e+01, 1.94108898e+01, ...,
         6.64320928e+00, 1.54422871e+01, 4.17991734e+00],
        ...,
        [7.16244548e-01, 2.77346969e+00, 1.37728321e+00, ...,
         3.51510326e+00, 2.54618342e+01, 2.35296378e-01],
        [1.42436485e-11, 5.73219146e-01, 9.30283397e+00, ...,
         5.85656441e+00, 1.74082997e+01, 2.78011610e+00],
        [4.53633502e-08, 2.89883131e-01, 9.19356593e+00, ...,
         4.63440869e+00, 1.75236343e+01, 2.14371611e+00]],
  mask=False,
  fill_value=1e+20)